In [200]:
# using MetadataTools,  DocStringExtensions
import Pkg;
Pkg.add("MLJ");
Pkg.add("DataFrames");
using Random: shuffle, MersenneTwister

# export MulticlassPerceptronClassifier, fit!, predict
using LinearAlgebra: mul!
using SparseArrays

import MLJBase
using MLJ
#using Revise
Pkg.add("CategoricalArrays")
#> needed for classifiers:
using CategoricalArrays

 Resolving package versions...
  Updating `~/Project.toml`
 [no changes]
  Updating `~/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/Project.toml`
 [no changes]
  Updating `~/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/Project.toml`
 [no changes]
  Updating `~/Manifest.toml`
 [no changes]


In [201]:
mutable struct MulticlassPerceptronClassifier <: MLJBase.Deterministic
    n_epochs::Int
    n_epoch_patience::Int
    f_average_weights::Bool
    f_shuffle_data::Bool
    element_type::DataType
end

In [202]:
# keyword constructor
function MulticlassPerceptronClassifier( ; 
                                        n_epochs=100,
                                        n_epoch_patience=5,
                                        f_average_weights=true,
                                        f_shuffle_data=false,
                                        element_type=Float32)

    model = MulticlassPerceptronClassifier(n_epochs,
                                           n_epoch_patience,
                                           f_average_weights,
                                           f_shuffle_data,
                                           element_type)
    
    message = MLJBase.clean!(model)
    isempty(message) || @warn message
    return model
end

MulticlassPerceptronClassifier

In [203]:
model = MulticlassPerceptronClassifier()

MulticlassPerceptronClassifier(n_epochs = 100,
                               n_epoch_patience = 5,
                               f_average_weights = true,
                               f_shuffle_data = false,
                               element_type = Float32,) @ 1…01

In [204]:
function MLJ.clean!(model::MulticlassPerceptronClassifier)
    warning = ""
    if model.n_epochs < 1
        warning *= "Need n_epochs ≥ 1. Resetting n_epochs=100 "
        model.n_epochs = 50
    end
    
    if model.n_epoch_patience <1
        warning *= "Need epoch_patience ≥ 1. Resetting epoch_patience=5 "
        model.epoch_patience = 5
    end

    return warning
end

In [205]:
mutable struct MulticlassPerceptronClassifierParameters{T}
    W::AbstractMatrix{T}
    b::AbstractVector{T}
    n_classes::Int
    n_features::Int
    is_sparse::Bool
end

In [206]:
function MulticlassPerceptronClassifierParameters(T::Type, n_classes::Int, n_features::Int, is_sparse::Bool) 
    
    if is_sparse==false
        return MulticlassPerceptronClassifierParameters{T}(rand(T, n_features, n_classes),
                                                                                       zeros(T, n_classes),
                                                                                       n_classes,
                                                                                       n_features,
                                                                                       is_sparse)
    else
        return  MulticlassPerceptronClassifierParameters{T}(sparse(rand(T, n_features, n_classes)),
                                                            spzeros(T, n_classes),
                                                            n_classes,
                                                            n_features,
                                                            is_sparse)     
    end
end

MulticlassPerceptronClassifierParameters

In [207]:
function MLJBase.fit(model::MulticlassPerceptronClassifier,
                     verbosity::Int,   
                     X,
                     y)
    
    #Xmatrix = MLJBase.matrix(X)
    n_classes    = length(unique(y))
    classes_seen = unique(y)
    n_features   = size(train_x,1)  # this assumes data comes in cols
    
    #decode  = MLJBase.decoder(y[1]) # for predict method
    decode =  false

    # Defining the fitpredict object
    is_sparse = issparse(X)
    perceptron = MulticlassPerceptronClassifierParameters(model.element_type, n_classes, n_features, is_sparse);
    
    ### Fitting code starts
    fit!(perceptron, X, y; 
         verbosity=verbosity, 
         n_epochs=model.n_epochs,
         f_average_weights=model.f_average_weights,
         f_shuffle_data=model.f_shuffle_data
        );   
    
    ### Fitting code ends
    cache = nothing
    fitresult = (perceptron, decode)
    report = NamedTuple{}()
    
    #> return package-specific statistics (eg, feature rankings,
    #> internal estimates of generalization error) in `report`, which
    #> should be a named tuple with the same type every call (can have
    #> empty values)
    return fitresult, cache, report
end

In [208]:
"""
Predicts the class for a given input in a `MulticlassPerceptronClassifier`.
The placeholder is used to avoid allocating memory for each matrix-vector multiplication.

- Returns the predicted class.
"""
function predict_with_placeholder(h::MulticlassPerceptronClassifierParameters, x::AbstractVector, class_placeholder::AbstractVector)
    #@fastmath class_placeholder .= At_mul_B!(class_placeholder, h.W, x) .+ h.b
    class_placeholder .= mul!(class_placeholder, transpose(h.W), x)  .+ h.b
    return argmax(class_placeholder)
end

predict_with_placeholder

In [209]:

"""
Compute the accuracy between `y` and `y_hat`.
"""
function accuracy(y::AbstractVector, y_hat::AbstractVector)
    acc = 0.
    @fastmath for k = 1:length(y)
            @inbounds  acc += y[k] == y_hat[k]
    end
    return acc/length(y_hat)
end

accuracy

In [210]:

"""


    average_flag=true:
        Maintains a running some of the average weight matrix W and bias vector b.

        One possible way to do this would be to update the weights after every example is seen  but this
        would be less efficient.
"""
function fit!(h::MulticlassPerceptronClassifierParameters, X::AbstractArray, y::AbstractVector;
              verbosity=0,
              n_epochs=50,
              learning_rate=0.1,
              f_average_weights=false,
              compute_accuracy=true,
              seed=MersenneTwister(1234),
              f_shuffle_data=false)
    
    n_features, n_samples = size(X)
    @assert length(y) == n_samples
    scores = []
    T = eltype(X)
    counter           = 0
    learning_rate     = T(learning_rate)
    class_placeholder = zeros(T, h.n_classes)
    y_preds           = zeros(Int16, n_samples)
    
    data_indices      = Array(1:n_samples)
    max_acc           = zero(T)
    best = 0
    
    if f_average_weights
        W_average =  zeros(T, h.n_features, h.n_classes)
        b_average =  zeros(T, h.n_classes)
    end

    @fastmath for epoch in 1:n_epochs

        n_mistakes = 0
        if f_shuffle_data
            shuffle!(seed, data_indices)
        end
        
        @inbounds for m in data_indices
            x     = view(X, :, m);
            y_hat = predict_with_placeholder(h, x, class_placeholder)
            
            if y[m] != y_hat
                n_mistakes += 1
                ####  wij ← wij − η (yj −tj) · xi
                h.W[:, y[m]]  .= h.W[:, y[m]]  .+ learning_rate .* x
                h.b[y[m]]      = h.b[y[m]]      + learning_rate
                h.W[:, y_hat] .= h.W[:, y_hat] .- learning_rate .* x
                h.b[y_hat]     = h.b[y_hat]     - learning_rate 
                
                if f_average_weights == true
                    counter_learning_rate = counter * learning_rate
                    W_average[:, y[m]]   .= W_average[:, y[m]]  .+ counter_learning_rate .* x
                    b_average[y[m]]       = b_average[y[m]]      + counter_learning_rate
                    W_average[:, y_hat]  .= W_average[:, y_hat] .- counter_learning_rate .* x
                    b_average[y_hat]      = b_average[y_hat]     - counter_learning_rate
                end
            end
            counter +=1
        end

        acc = (n_samples - n_mistakes)/n_samples
        # push!(scores, acc) maybe it would be nice to return an array with monitoring metrics to 
        # allow users to decide if the model has converged
        if acc > best
            best = acc
        end
    end
    
    if f_average_weights == true
        h.W .= h.W  .- W_average./counter 
        h.b .= h.b  .- b_average./counter
    end
    print("Best")
    print(best)
    
end

fit!

In [211]:
model = MulticlassPerceptronClassifier(n_epochs=50000; f_average_weights=false)

MulticlassPerceptronClassifier(n_epochs = 50000,
                               n_epoch_patience = 5,
                               f_average_weights = false,
                               f_shuffle_data = false,
                               element_type = Float32,) @ 1…00

In [212]:
model

MulticlassPerceptronClassifier(n_epochs = 50000,
                               n_epoch_patience = 5,
                               f_average_weights = false,
                               f_shuffle_data = false,
                               element_type = Float32,) @ 1…00

In [213]:
train_x = [0.100000000000000	1.10000000000000;
6.80000000000000	7.10000000000000;
-3.50000000000000	-4.10000000000000;
2	2.70000000000000;
4.10000000000000	2.80000000000000;
3.10000000000000	5;
-0.800000000000000	-1.30000000000000;
0.900000000000000	1.20000000000000;
5	6.40000000000000;
3.90000000000000	4;
7.10000000000000	4.20000000000000;
-1.40000000000000	-4.30000000000000;
4.50000000000000	0;
6.30000000000000	1.60000000000000;
4.20000000000000	1.90000000000000;
1.40000000000000	-3.20000000000000;
2.40000000000000	-4;
2.50000000000000	-6.10000000000000;
8.40000000000000	3.70000000000000;
4.10000000000000	-2.20000000000000;
-3	-2.90000000000000;
0.500000000000000	8.70000000000000;
2.90000000000000	2.10000000000000;
-0.100000000000000	5.20000000000000;
-4	2.20000000000000;
-1.30000000000000	3.70000000000000;
-3.40000000000000	6.20000000000000;
-4.10000000000000	3.40000000000000;
-5.10000000000000	1.60000000000000;
1.90000000000000	5.10000000000000;
-2	-8.40000000000000;
-8.90000000000000	0.200000000000000;
-4.20000000000000	-7.70000000000000;
-8.50000000000000	-3.20000000000000;
-6.70000000000000	-4;
-0.500000000000000	-9.20000000000000;
-5.30000000000000	-6.70000000000000;
-8.70000000000000	-6.40000000000000;
-7.10000000000000	-9.70000000000000;
-8	-6.30000000000000]

40×2 Array{Float64,2}:
  0.1   1.1
  6.8   7.1
 -3.5  -4.1
  2.0   2.7
  4.1   2.8
  3.1   5.0
 -0.8  -1.3
  0.9   1.2
  5.0   6.4
  3.9   4.0
  7.1   4.2
 -1.4  -4.3
  4.5   0.0
  ⋮        
 -5.1   1.6
  1.9   5.1
 -2.0  -8.4
 -8.9   0.2
 -4.2  -7.7
 -8.5  -3.2
 -6.7  -4.0
 -0.5  -9.2
 -5.3  -6.7
 -8.7  -6.4
 -7.1  -9.7
 -8.0  -6.3

In [214]:
train_x = transpose(train_x);

In [215]:
train_y = [1;
1;
1;
1;
1;
1;
1;
1;
1;
1;
2;
2;
2;
2;
2;
2;
2;
2;
2;
2;
3;
3;
3;
3;
3;
3;
3;
3;
3;
3;
4;
4;
4;
4;
4;
4;
4;
4;
4;
4]
print(length(y));

40

In [216]:
@time fitresult, _ , _  = MLJBase.fit(model, 1, train_x, train_y)

Best0.975  2.132636 seconds (14.68 M allocations: 387.177 MiB, 5.19% gc time)


((MulticlassPerceptronClassifierParameters{Float32}(Float32[3.13803 7.88323 2.00168 -11.3896; 4.36046 0.965389 6.72019 -9.60819], Float32[31.0001, 22.3, 29.3001, -82.5993], 4, 2, false), false), nothing, NamedTuple())